In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import numpy as np
import ast

import iqplot

import bokeh.io
import bokeh.models
import bokeh.plotting

bokeh.io.output_notebook()

In [ ]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

In [ ]:
Data = pd.read_csv('/Users/ulric/Desktop/Gated_arenas_Beh_manual_All.csv')
Data.head()

In [ ]:
Data = Data.explode('Gate') # wtf
Data['Gate'] = Data['Gate'].astype('string')
Data = Data.explode('Count') # wtf
Data['Count'] = Data['Count'].astype('float')


In [ ]:
sns.boxplot(x=Data.Behaviour, y = Data.Count, hue=Data.Training)
sns.stripplot(x=Data.Behaviour, y = Data.Count, hue=Data.Training, dodge=True)
#sns.catplot(x=Data.Behaviour, y = Data.Count,
        #    hue=Data.Training,
         #   col=Data.Gate)
plt.show

In [ ]:
g=sns.catplot(x='Behaviour', y = 'Count',
            hue='Training', showfliers = False,
            col='Gate', kind="box",
            data= Data)
g.map_dataframe(sns.stripplot,x='Behaviour', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

In [ ]:
g=sns.catplot(x='Behaviour', y = 'Count',
            hue='Training', showfliers = False,
            col='Gate', kind="box", row='Test',
            data= Data)
g.map_dataframe(sns.stripplot,x='Behaviour', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

plt.savefig('/Volumes/Universal_HD/VideoAnalysis/GatedArenas/Plots/Estimates/ManualPlot.png')

In [ ]:
g=sns.catplot(x='Behaviour', y = 'Count',
            hue='Training', showfliers = False,
            cut=0,
            col='Gate', kind="violin",
            data= Data)
g.map_dataframe(sns.stripplot,x='Behaviour', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

In [ ]:
Peekings = Data[Data.Behaviour == 'Peeking']

In [ ]:
Peekings

In [ ]:
Peekings_Left = Peekings[Data.Gate == 'Left']

Peekings_Left_Ctrl = Peekings_Left[Data.Training == 'Ctrl']

In [ ]:
Peekings_Left_Ctrl

In [ ]:
Set = Peekings_Left_Ctrl.Count
Set = np.asarray(Set)

In [ ]:
from scipy.stats import bootstrap



res = bootstrap(Set, statistic=np.mean ,confidence_level=0.9)
print(res.confidence_interval)

In [ ]:
Sorted=Data.groupby(['Behaviour'])

In [ ]:
np.mean(Data['Count'])

In [ ]:
Stat_Data = Data.groupby(['Behaviour', 'Training'])['Count']

Stat_Data.mean()

In [ ]:
ExData = Data[(Data.Training == 'Ctrl') & (Data.Behaviour == 'Peeking')]

Sample = ExData.Count

In [ ]:
rg = np.random.default_rng()

bs_sample = rg.choice(Sample, replace=True, size=len(Sample))

In [ ]:
bs_sample

In [ ]:
bs_replicate = np.mean(bs_sample)
bs_replicate

In [ ]:
# Number of replicatess
n_reps = 2000

# Initialize bootstrap replicas array
bs_reps_1975 = np.empty(n_reps)

# Compute replicates
for i in range(n_reps):
    bs_sample = rg.choice(Sample, size=len(Sample))
    bs_reps_1975[i] = np.mean(bs_sample)

In [ ]:
sns.ecdfplot(bs_reps_1975)

In [ ]:
def draw_bs_rep(data, func, rg):
    """Compute a bootstrap replicate from data."""
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)

def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
    """Sample bootstrap multiple times and compute confidence interval"""
    bs_reps = np.array(
        [draw_bs_rep(data, func, rg) for _ in range(n_reps)]
    )
    conf_int = np.percentile(bs_reps,[2.5, 97.5])
    return(conf_int)

In [ ]:
bs_reps_1975 = np.array(
    [draw_bs_rep(Sample, np.mean, rg) for _ in range(n_reps)]
)

conf_int_1975 = np.percentile(bs_reps_1975, [2.5, 97.5])
conf_int_1975

In [ ]:
conf_int_1975 = draw_bs_ci(Sample, np.mean,rg, n_reps=2000)
conf_int_1975

In [ ]:
Stat_Data=Data.groupby(['Behaviour', 'Training'])

Groups = list(Stat_Data.groups.keys())

In [ ]:
ConfInts = Stat_Data['Count'].apply(lambda x: draw_bs_ci(x))
Means = Stat_Data['Count'].mean()

In [ ]:
CIs=sns.catplot(x='Behaviour', y = 'Count',
            hue='Training', showfliers = False,
            cut=0,
            col='Gate', kind="violin",
            data= Data)
CIs.map_dataframe(sns.stripplot,x='Behaviour', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

In [ ]:
ConfInts.values

In [ ]:
Intervals = ConfInts.values

In [ ]:
#Categories = Means.axes[0]
Categories = []
for sublist in Means.axes:
    for item in sublist:
        Categories.append(item)


In [ ]:
X_axis = ['_'.join(tups) for tups in Categories]

In [ ]:
sns.pointplot(x=X_axis,y=Means.values, join = False)
sns.lineplot(x=X_axis, y=ConfInts.values)


In [ ]:
sns.lineplot(ConfInts.values[0])

In [ ]:
ConfInts.values[1]

In [ ]:
AutoData = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/AutoDataSet.csv')
AutoData.head()

In [ ]:
#AutoData.columns=AutoData.columns.droplevel(0)
Melted = pd.melt(AutoData, id_vars=['Training', 'Starvation', 'Date'], value_name = 'Count', value_vars=['Peeks Left', 'Peeks Right', 'Peeks Top', 'Long Peeks Left', 'Long Peeks Right', 'Long Peeks Top'], var_name='Gate')

Melted = Melted.loc[Melted['Starvation'] == 'Overnight no Water']

Melted['Beh'] = 'Value'
Melted['Side'] = 'Value'

In [ ]:
Melted['Beh'].loc[(Melted['Gate'].str.contains('Long'))] = 'Squeezing'
Melted['Beh'].loc[(Melted['Gate'].str.contains('Long')==False)] = 'Peeking'

Melted['Side'].loc[(Melted['Gate'].str.contains('Left'))] = 'Rewarded Side'
Melted['Side'].loc[(Melted['Gate'].str.contains('Right'))] = 'Punished Side'
Melted['Side'].loc[(Melted['Gate'].str.contains('Top'))] = 'Empty Side'

In [ ]:
sns.set(font_scale=1.5)


g=sns.catplot(x='Count', y = 'Side',
            hue='Training', showfliers = False,
            col='Beh',
            kind="box", #row='Starvation',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Count', y = 'Side',
            hue='Training', dodge=True,
            palette=["#404040"])
g.set(
      #xlabel="Relative time spent in 'rewarded' side",
      ylabel='',
      #yticks=['Rewarded side', 'Punished side', 'Empty side'],
      #xlim=(0,1),
      #xticks=np.arange(0,1,0.1),
      )

plt.text(15, 3,
         'n=9',
         horizontalalignment='center',
         verticalalignment='top',
         )
ax= g.axes
ax[0][0].set_title('Peeking')
ax[0][1].set_title('Squeezing')
#g._legend.remove()
plt.show

plt.savefig('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/AutoPlot_Old.png')

In [ ]:
ax[0][0].set_title('TheTest')

In [ ]:
TimeData = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/DataSetTest.csv')

TimeData.head()

In [ ]:
not_list_cols = [col for col in TimeData.columns if col not in ['Visits Left Corner', 'Durations Left Corner']] #list of columns you are not exploding (assume col1 and col2 are being exploded)
TimeData2 = TimeData.set_index(not_list_cols).apply(pd.Series.explode).reset_index()

#TimeData2 = TimeData.explode('Visits Left Corner')

#TimeData.explode(['Durations Left Corner', 'Visits Left Corner'])
TimeData2
#TimeMelted = pd.melt(Unpacked, id_vars=['Training', 'Starvation'], value_name = 'Timestamps', value_vars=['Visits Left Corner'], var_name='Gate')

In [ ]:
TimeData = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/DataSetTest.csv')
TimeData.head()

TimeData[['Visits Left Corner', 'Durations Left Corner',
          'Visits Right Corner', 'Durations Right Corner',
          'Visits Top Corner', 'Durations Top Corner',
          'Visits Left Front', 'Durations Left Front',
          'Visits Right Front', 'Durations Right Front',
          'Visits Top Front', 'Durations Top Front']] = TimeData[['Visits Left Corner', 'Durations Left Corner',
          'Visits Right Corner', 'Durations Right Corner',
          'Visits Top Corner', 'Durations Top Corner',
          'Visits Left Front', 'Durations Left Front',
          'Visits Right Front', 'Durations Right Front',
          'Visits Top Front', 'Durations Top Front']].applymap(ast.literal_eval)

TimeData = TimeData.explode(['Visits Left Corner', 'Durations Left Corner',]).reset_index().drop("index",1)\
    .explode(['Visits Right Corner', 'Durations Right Corner',]).reset_index().drop("index",1)\
    .explode(['Visits Top Corner', 'Durations Top Corner',]).reset_index().drop("index",1)\
    .explode(['Visits Left Front', 'Durations Left Front',]).reset_index().drop("index",1)\
    .explode(['Visits Right Front', 'Durations Right Front',]).reset_index().drop("index",1)\
    .explode(['Visits Top Front', 'Durations Top Front']).reset_index().drop("index",1)\



#TimeData.loc[TimeData.astype(str).drop_duplicates().index]

TimeData

In [ ]:
print(TimeData2['Visits Left Corner'][0])
print(TimeData2['Durations Left Corner'][0])

In [ ]:
TimeMelted = pd.melt(TimeData, id_vars=['Training', 'Starvation', 'Date', 'Fly'], value_name = 'Values',
                     value_vars=['Visits Left Corner', 'Durations Left Corner',
          'Visits Right Corner', 'Durations Right Corner',
          'Visits Top Corner', 'Durations Top Corner',
          'Visits Left Front', 'Durations Left Front',
          'Visits Right Front', 'Durations Right Front',
          'Visits Top Front', 'Durations Top Front'],
                     var_name="Variable")

TimeMelted[['Values',]] = TimeMelted[['Values',]].applymap(ast.literal_eval)
TimeMelted = TimeMelted.explode('Values').drop_duplicates()

In [ ]:
LeftCornerVisits = TimeMelted[TimeMelted['Variable'].isin(['Visits Left Corner', 'Durations Left Corner'])].reset_index().pivot_table(index = ['Fly', 'Training'], columns = 'Variable', values = 'Values').reset_index()

In [ ]:
#LeftCornerVisits.pivot(columns = 'Variable')['Values']
LeftCornerVisits=LeftCornerVisits.set_index(['Variable'])['Values'].unstack()
LeftCornerVisits

In [ ]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label='Visits time',
    y_axis_label='Visits duration'
)


p.circle(
    source= LeftCornerVisits[LeftCornerVisits['Training']== 'Trained'],
    x='Visits Left Corner',
    y='Durations Left Corner',
    legend_label = 'Trained flies'
)

p.circle(
    source=LeftCornerVisits[LeftCornerVisits['Training']== 'Ctrl'],
    x='Visits Left Corner',
    y='Durations Left Corner',
    color = 'orange',
    legend_label = 'Control flies'
)

bokeh.io.show(p)

In [ ]:
# First time related plot

Data = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/DataSetAugust22.csv')
Data.head()
Data = Data[Data['Test Starvation'] == "Overnight no Water"]
Data = Data[Data["Training Starvation"] == "Not starved"]  #.reset_index()


In [ ]:
Data['TimeSpentRewSide'] = 'i'

conditions = [
    ((Data["Reinforced_side"].str.contains("Left"))),
    ((Data["Reinforced_side"].str.contains("Right"))),
]
values = [
    Data['Relative Time Left'],
    Data['Relative Time Right'],
]
Data["TimeSpentRewSide"] = np.select(conditions, values)

#Data['TimeSpentRewSide'] = Data['Relative Time Left'] if Data['Reinforced_side'].str.contains('Left') else Data['Relative Time Right']

In [ ]:
sns.set_theme(font_scale=1.5)
sns.set(rc={'figure.figsize':(10, 15)})

g=sns.catplot(x='TimeSpentRewSide', y = 'Training',
            hue='Training', showfliers = False,
            #col='Gate',
            kind="box",
            #row='Test',
            data= Data)
g.set_xticklabels(size= 10)
g.map_dataframe(sns.stripplot,x='TimeSpentRewSide', y = 'Training',
            hue='Training', dodge=True,
            palette=["#404040"])
g.refline(x=0.5,
          color = "red",
          )
g.set(
      xlabel="Relative time spent in 'rewarded' side",
      ylabel='',
      #xlim=(0,1),
      #xticks=np.arange(0,1,0.1),
      )
plt.text(1, 1.5,
         'n=30',
         horizontalalignment='center',
         verticalalignment='top',
         )

plt.show
plt.savefig('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas/Results/TimeSpentLR.png')